In [2]:
import requests
import pandas as pd

url = "https://api.citybik.es/v2/networks/"
response = requests.get(url)


# confirm there is a good response from url request
if response.status_code == 200:
    data = response.json()

    #create empty variable to store found networks from Montreal
    montreal_network_id = None

    # Search for Montreal network by checking 'id' field
    #The location and name elements did not contain any hits for montreal although the map shows Montreal is supported
    # BIXI-Montreal showed up in 'ID' through links on CITYBIKE website and documentation API.

#Use network['id] to find hits for 'Montreal', search using converted lower case to avoid missing networks due to case sensitivity
    for network in data['networks']:
        if 'montreal' in network['id'].lower():
            montreal_network_id = network['id']
            break

# query using the network_id for Montreal, create url using montreal_network_id, 

    if montreal_network_id:
        montreal_url = f'http://api.citybik.es/v2/networks/{montreal_network_id}'
        montreal_response = requests.get(montreal_url)

#store response into montreal_data if sucessful request.
        if montreal_response.status_code == 200:
            montreal_data = montreal_response.json()

# Extract station information into a list of dictionaries
            station_data = []
            stations = montreal_data['network']['stations']
            for station in stations:
                station_data.append({
                    'name': station['name'],
                    'latitude': station['latitude'],
                    'longitude': station['longitude'],
                    'free_bikes': station['free_bikes'],
                    'empty_slots': station['empty_slots']
                })

# Create DataFrame from the collected data
            montreal_df = pd.DataFrame(station_data)

# Display the first few rows of the DataFrame

            print(montreal_df.head())
            print() 

# handle situations where unable to find network for montreal and display the error code
        else:
            print(f"Failed to retrieve Montreal data: {montreal_response.status_code}")
    else:
        print("Montreal network not found.")
else:
    print(f"Failed to retrieve networks data: {response.status_code}")

# Check for duplicate rows in dataframe
duplicates = montreal_df.duplicated()
duplicate_count = duplicates.sum()
print(f"Duplicate Count: {duplicate_count}")
print()
# Count null values in each column
nulls = montreal_df.isnull()
null_check = nulls.sum()
print(f"Null counts: {null_check}" )

#export to a CSV for part 2
montreal_df.to_csv('montreal_df.csv', index=False)



                                                name   latitude  longitude  \
0      Gare d'autocars de Montréal (Berri / Ontario)  45.516926 -73.564257   
1                                 Marché Maisonneuve  45.553219 -73.539782   
2      Bibliothèque d'Ahuntsic (Lajeunesse / Fleury)  45.553400 -73.662255   
3  Cité des Arts du Cirque (Paul Boutet / des Reg...  45.559842 -73.615447   
4                          Omer-Lavallée / du Midway  45.545776 -73.562175   

   free_bikes  empty_slots  
0           9            3  
1          14            8  
2           8            5  
3           8           26  
4           6           12  

Duplicate Count: 0

Null counts: name           0
latitude       0
longitude      0
free_bikes     0
empty_slots    0
dtype: int64
